# Generador de preguntas de Moodle

In [27]:
# Paquetes
import itertools
import os
import random

import numpy as np
from scipy.stats import poisson

## Funciones adicionales


In [28]:
def DistrPoisson(m, unaD):
    """
    Genera una expresión para calcular una probabilidad en una distribución de Poisson
    con un parámetro (P(X = a), P(X <= a) o P(X >= a)) o con dos parámetros
    (P(a <= X <= b)), y devuelve el enunciado junto con el resultado calculado.

    Parámetros:
    m: Media (parámetro lambda) de la distribución de Poisson.
    unaD: Booleano. Si es True, genera una desigualdad con un solo parámetro.
          Si es False, genera una desigualdad con dos parámetros.
    
    Retorna:
    Enunciado del problema y el resultado calculado.
    """

    # Definir la distribución de X
    distribucion = f'Sea \\( X \\sim \\Poi({m}) \\).'

    # Elegir dos valores aleatorios para los parámetros
    k1 = (m + 4 * np.sqrt(m) - 1) * random.random()
    k2 = (m + 4 * np.sqrt(m) - 1) * random.random()

    # Asignar valores enteros a y b
    a = int(min(k1, k2))
    b = int(max(k1, k2)) + 1

    # Seleccionar una desigualdad aleatoriamente
    D = random.random()

    # Generar enunciado y calcular resultado según el número de parámetros
    if unaD:
        if D < 0.6:
            desigualdad, resultado = ' = ', poisson.pmf(a, m)
        elif D < 0.8:
            desigualdad, resultado = r' \leq ', poisson.cdf(a, m)
        else:
            desigualdad, resultado = r' \geq ', 1 - poisson.cdf(a - 1, m)

        enunciado = f'{distribucion} Calcule \\( P(X{desigualdad}{a}) \\).'
    else:
        resultado = poisson.cdf(b, m) - poisson.cdf(a - 1, m)
        enunciado = f'{distribucion} Calcule \\( P({a} \\leq X \\leq {b}) \\).'

    return enunciado, round(resultado, 4)


## Parámetros y enunciado

In [29]:
# Nombre del cuestionario
name = "Poisson"

# Parámetros
par = ["m", "unaD"]
# Diccionario de parámetros
par_dict = {
    "m": list(range(1,11))*2,
    "unaD": [True, False],
}

# Enunciado
enunciado = r'''
\begin{numerical}[tolerance=0.001]%
    % - Indentificador
    {Poisson - [[id]]}
    % - Enunciado
    [[enunciado_generado]]
    \item[] [[ res ]]
\end{numerical}
'''

# Cantidad de preguntas a generar
n_preguntas = 25

# Código adicional a compilar antes de la generación de cada pregunta (opcional)
funciones = [
    "enunciado_generado, res = DistrPoisson(m, unaD)",
]
# Plantilla personalizada (opcional, por defento Plantilla.tex)
plantilla = "PlantillaAleph.tex"
# Carpeta de salida (opcional, por defecto output/)
# carpeta = "output/"

## Generar preguntas

In [30]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(18)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
if len(_par_comb) < _total:
    _total = len(_par_comb)
else:
    _par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

# Mensaje
print(_quiz)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.001]%
    % - Indentificador
    {Poisson - 1}
    % - Enunciado
    Sea \( X \sim \Poi(6) \). Calcule \( P(2 \leq X \leq 9) \).
    \item[] 0.8987
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.001]%
    % - Indentificador
    {Poisson - 2}
    % - Enunciado
    Sea \( X \sim \Poi(4) \). Calcule \( P(9 \leq X \leq 10) \).
    \item[] 0.0185
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.001]%
    % - Indentificador
    {Poisson - 3}
    % - Enunciado
    Sea \( X \sim \Poi(5) \). Calcule \( P(X \geq 0) \).
    \item[] 1.0
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.001]%
    % - Indentificador
    {Poisson - 4}
    % - Enunciado
    Sea \( X \sim \Poi(1) \). Calcule \( P(0 \leq X \leq 3) \).
    \item[] 0.981
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[toleran

## Generar .tex, .pdf y .xml

In [31]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(18)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
if len(_par_comb) < _total:
    _total = len(_par_comb)
else:
    _par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

_parametros_latex = ''
for _parametro in par:
    _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\\{' + ', '.join(map(str, par_dict[_parametro])) + '\\}$' + '\n'

# Leo el archivo plantilla
# Si la variable plantilla no está definida, la defino como 'Plantilla.tex'
if not 'plantilla' in locals():
    plantilla = 'Plantilla.tex'
_plantilla = open(plantilla, 'r', encoding='utf-8')
_t_plantilla = _plantilla.read()
_plantilla.close()
# Reemplazo
_t_plantilla = _t_plantilla.replace('{{QUIZ}}', _quiz)
_t_plantilla = _t_plantilla.replace('{{Cuestionario}}', name)
_t_plantilla = _t_plantilla.replace('{{Parámetros}}', _parametros_latex)
_t_plantilla = _t_plantilla.replace('{{Número de preguntas}}', str(id-1))

# Escribo el archivo
# Si la variable carpeta no está definida, la defino como 'test'
if not 'carpeta' in locals():
    carpeta = 'output'
# Si no existe la carpeta, la creo
if not os.path.exists(carpeta):
    os.makedirs(carpeta)
_examen = open(f'{carpeta}/{name}.tex', 'w', encoding='utf-8')
_examen.write(_t_plantilla)
_examen.close()

# Compilo el examen con xeLaTeX
os.system(f'latexmk -xelatex "{name}.tex" -output-directory="./{carpeta}"')

# Elimino todos los archivos llamados examen, salvo el pdf y el xml
_archivos = os.listdir(f'./{carpeta}')
for _archivo in _archivos:
    if name in _archivo:
        if '.pdf' not in _archivo and '.xml' not in _archivo and '.tex' not in _archivo:
            try:
                os.remove(f"{carpeta}/"+_archivo)
            except:
                pass
# Elimino los archivos fls
for _archivo in _archivos:
    if '.fls' in _archivo:
        try:
            os.remove(_archivo)
        except:
            pass

# Mensaje
print(f'Examen generado con éxito, se generaron {id-1} preguntas.')

Examen generado con éxito, se generaron 25 preguntas.
